In [1]:
import numpy  as np
import pandas as pd 

# A Multiply Indexed Series

### The bad way

In [2]:
# We assume a list of tuple
index = [('California', 2000), ('California', 2010),
        ('New York', 2000), ('New York', 2010),
        ('Texas', 2000), ('Texas', 2010)]

populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]

In [3]:
pop = pd.Series(data=populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

With this indexing scheme, you can straightforwardly index or slice the series based
on this multiple index:

In [6]:
pop[("California", 2000) : ("New York", 2000)]

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
dtype: int64

### The better way: Pandas MultiIndex

In [11]:
index = pd.MultiIndex.from_tuples(index)
pop = pd.Series(data=populations, index= index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [14]:
pop.index.values

array([('California', 2000), ('California', 2010), ('New York', 2000),
       ('New York', 2010), ('Texas', 2000), ('Texas', 2010)], dtype=object)

Here the first two columns of the Series representation show the multiple index val‐
ues, while the third column shows the data. Notice that some entries are missing in
the first column: in this multi-index representation, any blank entry indicates the
same value as the line above it.

In [19]:
pop[:,2000]

California    33871648
New York      18976457
Texas         20851820
dtype: int64

The result is a singly indexed array with just the keys we’re interested in. This syntax
is much more convenient (and the operation is much more efficient!) than the home-
spun tuple-based multi-indexing solution that we started with. We’ll now further dis‐
cuss this sort of indexing operation on hierarchically indexed data.

## MultiIndex as extra dimension

In [21]:
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [23]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


Seeing this, you might wonder why would we would bother with hierarchical indexing at all. The reason is simple: just as we were able to use multi-indexing to represent
    two-dimensional data within a one-dimensional Series , we can also use it to repre‐
sent data of three or more dimensions in a Series or DataFrame . Each extra level in a
multi-index represents an extra dimension of data; taking advantage of this property
gives us much more flexibility in the types of data we can represent. Concretely, we
might want to add another column of demographic data for each state at each year
(say, population under 18); with a MultiIndex this is as easy as adding another col‐
umn to the DataFrame :

In [26]:
pop_df = pd.DataFrame({"total" : pop, "under18" : [9267089, 9284094,
                                                    4687374, 4318033,
                                                    5906301, 6879014]})

pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [28]:
# Here we compute the fraction of people under 18 by year, given the above data
df_fract_u18 = pop_df["under18"] / pop_df["total"]

df_fract_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


# Methods of MultiIndex Creation

In [33]:
index = pd.MultiIndex.from_product([["a","b"],[1,2]])
df = pd.DataFrame(np.random.rand(4,2),index=index,columns=["data1","data2"])

df

data1     data2
a 1  0.488947  0.474227
  2  0.107693  0.193589
b 1  0.285973  0.928089
  2  0.581354  0.809186

The work of creating the MultiIndex is done in the background.

Similarly, if you pass a dictionary with appropriate tuples as keys, Pandas will auto‐
matically recognize this and use a MultiIndex by default:

In [35]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}

pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

## Explicit MultiIndex constructors

In [42]:
# from_array() method
index = pd.MultiIndex.from_arrays([["a","a","b","b"],[1,2,1,2]])

pd.DataFrame(np.random.rand(4,2),index=index)

0         1
a 1  0.454650  0.979954
  2  0.772260  0.139894
b 1  0.339143  0.061188
  2  0.949991  0.659528

In [45]:
index = pd.MultiIndex.from_tuples([("a",1),("a",2),("b",1),("b",2)])
pd.DataFrame(np.random.rand(4,2),index=index)

0         1
a 1  0.804060  0.332096
  2  0.614593  0.816478
b 1  0.132542  0.730390
  2  0.143162  0.806013

In [46]:
index = pd.MultiIndex.from_product([["a","b"],[1,2]])
pd.DataFrame(np.random.rand(4,2),index=index)

0         1
a 1  0.113519  0.117397
  2  0.558381  0.019667
b 1  0.813862  0.025812
  2  0.509702  0.951250

## MultiIndex level names

In [56]:
df_pop = pd.DataFrame(pop)
df_pop

0
state      year          
California 2000  33871648
           2010  37253956
New York   2000  18976457
           2010  19378102
Texas      2000  20851820
           2010  25145561

In [66]:
df_pop.index.names = ["state","year"]
df_pop

0
state      year          
California 2000  33871648
           2010  37253956
New York   2000  18976457
           2010  19378102
Texas      2000  20851820
           2010  25145561

## MultiIndex for columns

In [67]:
# hierarchical indices and columns

index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37
# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      59.0  35.7  41.0  36.7  37.0  38.0
     2      56.0  37.4  45.0  38.1  48.0  39.1
2014 1      31.0  35.9  44.0  37.5  49.0  37.5
     2      50.0  37.0  40.0  37.1  41.0  35.4

# Indexing and Slicing a MultiIndex

## Multiply indexed Series

In [69]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [71]:
pop["California",2010]

37253956

In [72]:
pop["California"]

year
2000    33871648
2010    37253956
dtype: int64

Partial slicing is available as well, as long as the MultiIndex is sorted

In [74]:
pop.loc["California", 2010]

37253956

With sorted indices, we can perform partial indexing on lower levels by passing an
empty slice in the first index

In [76]:
pop[:,2010]

state
California    37253956
New York      19378102
Texas         25145561
dtype: int64

selection based on Boolean masks:

In [80]:
pop[pop > 25000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

Selection based on fancy indexing also works:

In [94]:
pop[["California","Texas"]][:,2010]

state
California    37253956
Texas         25145561
dtype: int64

## Multiply indexed DataFrames

In [96]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      59.0  35.7  41.0  36.7  37.0  38.0
     2      56.0  37.4  45.0  38.1  48.0  39.1
2014 1      31.0  35.9  44.0  37.5  49.0  37.5
     2      50.0  37.0  40.0  37.1  41.0  35.4

In [99]:
health_data["Guido","HR"]

year  visit
2013  1        41.0
      2        45.0
2014  1        44.0
      2        40.0
Name: (Guido, HR), dtype: float64

In [117]:
health_data.iloc[:2,:2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      59.0  35.7
     2      56.0  37.4

if we try to create a slice a tuple like `health_data[(:,1),(:"HR")]` will lead to a syntax err :`SyntaxError: invalid syntax`

You could get around this by building the desired slice explicitly using Python’s built-
in slice() function, but a better way in this context is to use an IndexSlice object,
which Pandas provides for precisely this situation. For example:

In [123]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      59.0  35.7  41.0  36.7  37.0  38.0
     2      56.0  37.4  45.0  38.1  48.0  39.1
2014 1      31.0  35.9  44.0  37.5  49.0  37.5
     2      50.0  37.0  40.0  37.1  41.0  35.4

In [127]:
# Assumption, we want to get first visit of Guido and Sue in HR
idx = pd.IndexSlice

health_data.loc[idx[:,1],idx["Guido":"Sue","HR"]]

,subject,Guido,Sue
,type,HR,HR
year,visit,,
2013,1,41.0,37.0
2014,1,44.0,49.0


# Rearranging Multi-Indices

One of the keys to working with multiply indexed data is knowing how to effectively
transform the data. There are a number of operations that will preserve all the infor‐
mation in the dataset, but rearrange it for the purposes of various computations. We
saw a brief example of this in the stack() and unstack() methods, but there are
many more ways to finely control the rearrangement of data between hierarchical
indices and columns, and we’ll explore them here.

## Sorted and unsorted indices

In [131]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ["char", "int"]
data

char  int
a     1      0.336750
      2      0.019113
c     1      0.683209
      2      0.243134
b     1      0.197476
      2      0.401603
dtype: float64

In [135]:
data = data.sort_index()
data["a":"b"]

char  int
a     1      0.336750
      2      0.019113
b     1      0.197476
      2      0.401603
dtype: float64

# Data Aggregations on Multi-Indices

In [136]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      59.0  35.7  41.0  36.7  37.0  38.0
     2      56.0  37.4  45.0  38.1  48.0  39.1
2014 1      31.0  35.9  44.0  37.5  49.0  37.5
     2      50.0  37.0  40.0  37.1  41.0  35.4

In [139]:
data_mean  = health_data.mean(level="year")
data_mean

subject   Bob        Guido         Sue       
type       HR   Temp    HR  Temp    HR   Temp
year                                         
2013     57.5  36.55  43.0  37.4  42.5  38.55
2014     40.5  36.45  42.0  37.3  45.0  36.45

In [141]:
data_mean_visit = health_data.mean(level="visit")
data_mean_visit

subject   Bob       Guido         Sue       
type       HR  Temp    HR  Temp    HR   Temp
visit                                       
1        45.0  35.8  42.5  37.1  43.0  37.75
2        53.0  37.2  42.5  37.6  44.5  37.25

In [146]:
data_mean_type = health_data.mean(axis=1,level="type")
data_mean_type

type               HR       Temp
year visit                      
2013 1      45.666667  36.800000
     2      49.666667  38.200000
2014 1      41.333333  36.966667
     2      43.666667  36.500000